## Writeup Template 

### Vehicle Detection Project

The steps of this project are the following:

* Get the training data: we need images of cars (positive samples) and images representing something else (negative samples).
* Feature extraction (for each sample of the training set):
  * Perform a Histogram of Oriented Gradients (HOG) feature extraction
  * Extract binned color features, as well as histograms of color
  * Concatenate the previous results in a vector and normalize
* Train a classifier ( here i used both a linear SVM and MPL classifier)
* Implement a sliding-window technique and use the trained classifier to search for vehicles in images.
* Run the pipeline on a video stream and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
* Estimate a bounding box for vehicles detected.
[//]: # (Image References)
[image1]: ./examples_for_writeup/car&notcar_hog.png
[image2]: ./examples_for_writeup/scale1.png
[image3]: ./examples_for_write_up/scale1.5.png
[image4]: ./examples_for_write_up/scale2.png
[image4]: ./examples_for_write_up/car_position.png
[image2]: ./examples_for_writeup/mlpscale1.png
[image3]: ./examples_for_write_up/mlpscale1.5.png
[image4]: ./examples_for_write_up/mlpscale2.png
[image4]: ./examples_for_write_up/mlpcar_position.png


### Training data

I used the data provided by Udacity. Here are links to the labeled data for [vehicle](https://s3.amazonaws.com/udacity-sdc/Vehicle_Tracking/vehicles.zip) and [non-vehicle](https://s3.amazonaws.com/udacity-sdc/Vehicle_Tracking/non-vehicles.zip). These example images come from a combination of the [GTI vehicle image database](http://www.gti.ssr.upm.es/data/Vehicle_database.html), the [KITTI vision benchmark suite](http://www.cvlibs.net/datasets/kitti/), and examples extracted from the project video itself.

The dataset contains in total 17,760 color images of dimension 64×64 px. 8,792 samples contain a vehicle and 8,968 samples do not.

first,I started by reading in all the `vehicle` and `non-vehicle` images.then I applied bin_spatial and color_hist functions
These are the features I used in this project:
* Spatial features: a down sampled copy of the image
* Color histogram features that capture the statistical color information of each image. Cars often have very saturated colors while the background has a pale color. This feature could help to identify the car by the color information.
* Histogram of oriented gradients (HOG): that capture the gradient structure of each image channel and work well under different lighting conditions.

I then explored different color spaces and different `skimage.hog()` parameters (`orientations`, `pixels_per_cell`, and `cells_per_block`).  I grabbed random images from each of the two classes and displayed them to get a feel for what the `skimage.hog()` output looks like.

Here is an example using the `YCrCb` color space and HOG parameters of `orientations=9`, `pixels_per_cell=(8, 8)` and `cells_per_block=(2, 2)`:


<img src="./examples_for_writeup/car&notcar_hog.png" >

the following step is to extract features, normalize data,create labels vector,shuffle and split the data into a training and testing set,
and then train the classifier, i used the SVM classifier.
after tuning parameters several times, I got 99% accuracy and the confusion matrix shows 14 fault-positive with these:
- color_space = 'YCrCb'
- orient = 9
- pix_per_cell = 8
- cell_per_block = 2
- hog_channel = 'ALL'
- spatial_size = (32,32)
- hist_bins = 16
Then i tried to get the best c parameter using GridSearchCV, but i got again the same accuracy (99%),and more fault-positive, so i ignore it,
after that i used MPL Classifier, the accuracy this time was better 99.4% with less fault-positive.

### Sliding Window Search

We have to deal now with images coming from a front-facing camera on a car. We need to extract from these full-resolution images some sub-regions and check if they contain a car or not. To extract subregions of the image I used a sliding window approach. It is important to minimize the number of subregions used to improve the performance and to avoid looking for cars where they cannot be (for example in the sky).

For each subregion, we need to compute the feature vector and feed it to the classifier. The classifier SVM/MPL, will predict if there is a car or not in the images.


## Test on images

The function find_cars can both extract features and make predictions by computing the HOG transform only once for the entire picture. The HOG is then sub-sampled to get all of its overlaying windows. This function is called three times at different scale: 1, 1.5 and 2. I recorded the positions of positive detections in each frame of the video. From the positive detections, I created a heat map and then thresholded it to identify vehicle positions. I then used scipy.ndimage.measurements.label() to identify individual blobs in the heat map. I then assumed each blob corresponded to a vehicle. Finally, I constructed bounding boxes to cover the area of each blob detected using draw_labeled_boxes function
One thing that really helped was adding deifferent threshold for both classifier, which helps to ensure high confidence predictions. In fact, by using the threshold you can ensure that you are only considering high confidence predictions as vehicle detections.
for SVM I used decision function as threshold, and for MLP I used the difference between real features and predicted features.
here are some example images:

### Using SVM with multiple scale:

<img src="./examples_for_writeup/scale1.png" >

<img src="./examples_for_writeup/scale1.5.png" >

<img src="./examples_for_writeup/scale2.png" >

than the final result after using all scales is as shown below, we can notice that in the bottom left of the image a detected car in the oppsite road (it s not a fault positive).

<img src="./examples_for_writeup/car_position.png" >

### Using MLP with multiple scale:

<img src="./examples_for_writeup/mlpscale1.png" >

<img src="./examples_for_writeup/mlpscale1.5.png" >

<img src="./examples_for_writeup/mlpscale2.png" >

than the final result after using all scales is as shown below, we can notice that in the bottom left of the image a detected car in the opposite road (it s not a fault positive).with scale 1.5 there is two fault-positive,but again it s because the treshold i m using for MLP Classifier is not not strong enough to remove all fault-positive.

<img src="./examples_for_writeup/mlpcar_position.png" >

## Video implementation using MLP Classifier


Finally, I tested the pipeline on a video stream,by creating a process function that pass all images through the pipeline process.
Here is the result of the process function in the following video


 
Click [here](https://youtu.be/yKROjVC3f34) to see the complete video.

## Video implementation using SVC


   
Click [here](https://youtu.be/NN4lW5A6EW4) to see the complete video.

## Discussion

it seems that using the HOG and the SVM/MLP classifier works quite well for the tested images and videos, but it turned out to be very slow,i think that a CNN approach would be probably better for real word applications, because detectors based on CNN are faster, more accurate and more robust. However, it has to be said that this is not a fair comparison since that the deep learning is using the GPU and the first approach is using CPU.
